# PCML Project-1 ~ Team #60

In [28]:
# Useful starting lines
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import datetime

from gradient_descent import gradient_descent
from stochastic_gradient_descent import stochastic_gradient_descent
from least_squares import least_squares
from ridge_regression import ridge_regression
from costs import compute_loss

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [40]:
from proj1_helpers import *
from helpers import *

DATA_TRAIN_PATH = "../Data/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

tX, mean_x, std_x = standardize(tX, mean_x=None, std_x=None)

print(tX.shape)

(250000, 31)


## Linear regression using gradient descent

In [63]:
# from gradient_descent import *
# from plots import gradient_descent_visualization

# Define the parameters of the algorithm.
max_iters = 1000
gamma = 0.01

# Initialization
w_initial = np.zeros(tX.shape[1])

# Start gradient descent.
# start_time = datetime.datetime.now()
gradient_losses, gradient_ws = gradient_descent(y, tX, w_initial, max_iters, gamma)
# end_time = datetime.datetime.now()

print('parameters: ',gradient_ws[-1])


Gradient Descent(0/999): loss=0.5
Gradient Descent(1/999): loss=0.4930403211757793
Gradient Descent(2/999): loss=0.4870098501022072
Gradient Descent(3/999): loss=0.4817168268664753
Gradient Descent(4/999): loss=0.4770128047663221
Gradient Descent(5/999): loss=0.4727827104481938
Gradient Descent(6/999): loss=0.4689371916527971
Gradient Descent(7/999): loss=0.4654067258061777
Gradient Descent(8/999): loss=0.4621370840030196
Gradient Descent(9/999): loss=0.45908583830059513
Gradient Descent(10/999): loss=0.45621967211044767
Gradient Descent(11/999): loss=0.4535123087925472
Gradient Descent(12/999): loss=0.45094291613495796
Gradient Descent(13/999): loss=0.4484948771748208
Gradient Descent(14/999): loss=0.4461548430418665
Gradient Descent(15/999): loss=0.4439120029218537
Gradient Descent(16/999): loss=0.4417575211821269
Gradient Descent(17/999): loss=0.4396841032046329
Gradient Descent(18/999): loss=0.43768566032584444
Gradient Descent(19/999): loss=0.43575705109814233
Gradient Descent(20/

## Linear regression using stochastic gradient descent

In [61]:
# Define the parameters of the algorithm.
max_iters = 1000
gamma = 0.01
batch_size = 50

# Initialization
w_initial = np.zeros(tX.shape[1])

# Start SGD.
# start_time = datetime.datetime.now()
stoch_gradient_losses, stoch_gradient_ws = stochastic_gradient_descent(y, tX, w_initial, batch_size, max_iters, gamma)
# end_time = datetime.datetime.now()

print(stoch_gradient_ws[-1])

Stochastic Gradient Descent(0/999): loss=0.5
Stochastic Gradient Descent(1/999): loss=0.49097577769773404
Stochastic Gradient Descent(2/999): loss=0.4787077336190901
Stochastic Gradient Descent(3/999): loss=0.48202057420798816
Stochastic Gradient Descent(4/999): loss=0.4801342403316257
Stochastic Gradient Descent(5/999): loss=0.4857553603991821
Stochastic Gradient Descent(6/999): loss=0.4898959193756118
Stochastic Gradient Descent(7/999): loss=0.4459445011035137
Stochastic Gradient Descent(8/999): loss=0.4617479985429819
Stochastic Gradient Descent(9/999): loss=0.4541295788246645
Stochastic Gradient Descent(10/999): loss=0.44883650469220304
Stochastic Gradient Descent(11/999): loss=0.45098555849902766
Stochastic Gradient Descent(12/999): loss=0.42430456118339266
Stochastic Gradient Descent(13/999): loss=0.48637765682007533
Stochastic Gradient Descent(14/999): loss=0.43396794229063546
Stochastic Gradient Descent(15/999): loss=0.4396159616559118
Stochastic Gradient Descent(16/999): loss=

## Least squares regression using normal equations

In [56]:
# start_ls_time = datetime.datetime.now()
ls_wopt, ls_loss = least_squares(y,tX)
print(ls_loss)
print(ls_wopt)
# end_ls_time = datetime.datetime.now()

0.339686809908
[  8.03908656e-05  -7.20112725e-03  -6.05470621e-03  -5.47503706e-04
  -1.93917581e-02   4.73466159e-04  -2.60377150e-02   3.25108506e-01
  -3.80835171e-05  -2.72729929e+00  -2.21218331e-01   9.50817492e-02
   6.40392496e-02   2.73555898e+00  -3.31801082e-04  -9.54327750e-04
   2.74031566e+00  -5.34164922e-04   9.73498609e-04   3.69225052e-03
   3.54487428e-04  -5.43344599e-04  -3.30448035e-01  -1.40800497e-03
   8.31432882e-04   1.02117272e-03  -1.68047416e-03  -5.83664815e-03
  -1.11087997e-02   2.72775918e+00]


## Ridge regression using normal equations

In [64]:
# lambdas = np.logspace(-3, 1, 10)      
# φ_x = build_poly(x, degree)
# x_train, x_test, y_train, y_test = split_data(tX, y, ratio, seed)
    
#     for lamb in lambdas:

w_ridge = ridge_regression(y, tX, 0.01)

print(w_ridge)

# rmse_tr = np.sqrt(2*compute_loss(y, tX, w_ridge))
# rmse_te = np.sqrt(2*compute_loss(y, tX, w_ridge))


[-0.30849412  0.03572402 -0.24077855 -0.21610016 -0.01054923 -0.01919922
  0.34814992 -0.03018983  0.23504977 -0.01013674 -0.00087247 -0.15674991
  0.11455458 -0.02161131  0.18503225 -0.00076269 -0.00130455  0.23850622
 -0.00086979  0.00249252  0.10383837  0.00113401 -0.06198121 -0.14991608
  0.03645409  0.04367213  0.0436772  -0.02328488 -0.02309077 -0.0234825
 -0.09703028]


## Logistic regression using gradient descent or SGD

## Regularized logistic regression using gradient descent or SGD

## Generate predictions and save ouput in csv format for submission:

In [11]:
DATA_TEST_PATH = '/home/sarigian/Desktop/project1/Data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)